In [9]:
import os
import pickle

In [10]:
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

In [11]:
from conflab.data_loading.pose import ConflabPoseExtractor, ConflabToKinetics
from conflab.data_loading.labels import ConflabLabelExtractor
from conflab.data_loading.person import ConflabDataset, ConflabSubset
from conflab.constants import processed_pose_path, processed_ss_path

In [47]:
accel_proba = pd.read_csv('../accel/outputs/minirocket_accel.csv', header=None).to_numpy()
pose_logits = pickle.load(open('../pose/outputs/scores_5e.pkl', 'rb'))

In [48]:
(accel_proba.shape, pose_logits.shape)

((42882, 1), (42882, 2))

In [38]:
pose_extractor = ConflabPoseExtractor(processed_pose_path)
pose_extractor.load_from_pickle('../tracks.pkl')
label_extractor = ConflabLabelExtractor(os.path.join(processed_ss_path, 'speaking'))

In [39]:
# make windowed examples using the pose tracks.
examples = pose_extractor.make_examples()
# compose the dataset
dataset = ConflabDataset(examples, {
    'pose': pose_extractor,
    'label': label_extractor
}, transform=ConflabToKinetics())

100%|██████████| 8/8 [00:00<00:00, 505.85it/s]


In [40]:
labels = dataset.get_all_labels()

In [41]:
# sanity check AUCS
roc_auc_score(labels, accel_proba)

0.8127529000170967

In [42]:
# sanity check AUCS
roc_auc_score(labels, pose_logits[:,1])

0.6243771358652327

### Late fusion
This code performs score level fusion on the outputs of pose and accel classifiers

We reproduce the same folds as for the independent accel and pose models

In [57]:
def do_run(X, Y, random_state):
    pids = set(dataset.get_groups())
    pid_splits = KFold(n_splits=10, random_state=random_state, shuffle=True).split(range(len(pids)))
    aucs = []
    accs = []
    for f, (train_pids, test_pids) in enumerate(pid_splits):
        train_idx = [i for i, e in enumerate(dataset.examples) if e[0] in train_pids]
        test_idx = [i for i, e in enumerate(dataset.examples) if e[0] in test_pids]

        X_train = X[train_idx, :]
        Y_train = Y[train_idx]

        X_test = X[test_idx, :]
        Y_test = Y[test_idx]

        lr = LogisticRegression(C=1000000000).fit(X_train, Y_train)
        proba = lr.predict_proba(X_test)
        pred = (proba[:,1] > 0.5).astype(int)

        fold_acc = (pred == Y_test).sum() / len(Y_test)
        fold_auc = roc_auc_score(Y_test, proba[:,1])
        aucs.append(fold_auc)
        accs.append(fold_acc)
    return np.mean(aucs), np.mean(accs)

In [58]:
# pose-only score (for verification)
# auc should be the same as measured for pose model
X = pose_logits[:,1][:,None]
Y = labels
do_run(X,Y,random_state=22)

(0.6761154577391613, 0.7058173135458581)

In [59]:
# fusion score
X = np.hstack([accel_proba, pose_logits[:,1][:,None]])
Y = labels
do_run(X,Y,random_state=22)

(0.8233193271768162, 0.7748303737119817)